In [1]:
from thermoclassifier.phases.net import PhaseClassifier
from thermoclassifier.dataset.dataset_creator import *
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt

In [21]:
def epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len):
    epoch_losses = np.zeros([len(train_loader), ])
    correct = 0
    incorrect = 0
    
    for i, d in enumerate(train_loader):
        # Scale inputs and get predictions
        inp = d[:, :, :-1]
        inp[:, :, 0] /= 1000
        predictions = net(inp.float()).squeeze()
        
        # Each batch consists of measurement batches, where seq_len measurements are put into one batch. In such a 
        # measurement batch, every measurement has the same label as it needs to be from the same element an phase. 
        # This leads to a target array where the target class is contained for each measurement in the measurement 
        # batch. With this, CrossEntropyLoss would not work as the predictions are made for the whole measurement 
        # batch and CEL therefore expects only on class label per measurement batch. Therefore, only the first 
        # element of the last dimension of d is considered as target (all the entries in the last dimension are the 
        # same anyways so it could be any entry)
        targets = d[:, :, -1][:, 0].long()
                
        correct += (predictions.argmax(dim=-1) == targets).sum().item()
        incorrect += len(targets) - (predictions.argmax(dim=-1) == targets).sum().item()
        
        #print(predictions.argmax(dim=-1), targets)
        
        # Calculate the loss
        loss = loss_func(predictions, targets)
        epoch_losses[i] = loss
        
        # Backward step
        net.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Training accuracy: ', correct/(correct + incorrect))
    
    return epoch_losses.mean()

In [82]:
# Training and testing routines

def train(net, train_loader, batch_size, seq_len, save=True):
    # Hyperparameters
    nr_epochs = 5000
    lr = 0.001
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(net.parameters(), lr=lr)
    
    losses = np.zeros([nr_epochs, ])
    
    # Save the net with the lowest training loss as the best net
    best_net = net
    best_loss = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len)
    
    for i in range(nr_epochs):
        losses[i] = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len)
        
        if losses[i] < best_loss:
            best_net = net
        
        if i % 10 == 0:
            print(losses[i])
            
    return best_net
        
def test(net, test_loader): 
    correct = 0
    incorrect = 0

    for d in test_loader:
        inp = d[:, :, :-1]
        inp[:, :, 0] /= 1000
        prediction = net(inp.float()).argmax()
        target = d[:, :, -1][:, 0].long()
        if prediction == target:
            correct += 1
        else:
            incorrect += 1
        
        print(inp)
        print(prediction.argmax(dim=-1), target)
        print()

    accuracy = correct/(correct + incorrect)
    print('Test accuracy: ', accuracy)
    
    return accuracy

In [12]:
# Hyperparameters
seq_len = 1
batch_size = 32

# Create the dataset
measurement = 'C'
dc = DatasetCreator(elements=None, splits=(0.8, 0.2), validation=False, seq_len=seq_len, measurement=measurement, user='phase')
train_dataset, test_dataset, val_dataset = dc.get_datasets()

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
if val_dataset:
    val_loader = DataLoader(val_dataset)
    
# Create the network
t = True
net = PhaseClassifier(train=t, measurement=measurement)

Dataset shape:  (102693, 1, 4)


In [84]:
#Train the network
best_net = train(net, train_loader, batch_size, seq_len)

# Test the trained network
test(best_net, test_loader)

Training accuracy:  0.9021160156972724
Training accuracy:  0.9046380960727606
0.2477551534389671
Training accuracy:  0.90330402266951


KeyboardInterrupt: 

In [85]:
test(net, test_loader)

tensor([[[ 0.4580, 13.5332,  9.0000]]], dtype=torch.float64)
tensor(0) tensor([13])

tensor([[[ 0.3530, 32.4349, 15.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.2910, 39.5415,  6.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.9390, 59.4526, 75.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.6930, 44.2249, 57.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.6510, 36.3575, 70.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.6290, 36.7774, 75.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.9870, 31.3195, 17.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.4950, 27.8558, 67.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 0.9880, 27.3339, 43.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.9110, 30.6953, 17.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.5900, 29.4875, 11.0000]]], dtype=torch.float64)
tensor(0) tens

tensor(0) tensor([0])

tensor([[[ 1.9630, 39.1990, 67.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.0840, 32.7927, 40.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 0.4360, 29.0255, 22.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.5300, 30.5392, 29.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.4820, 27.1995, 25.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.2530, 27.4339, 59.0000]]], dtype=torch.float64)
tensor(0) tensor([7])

tensor([[[ 1.5180, 41.8400,  2.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.2060, 34.1353, 20.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 0.7780, 30.2630, 18.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.3970, 35.3859,  6.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.5250, 36.3005, 74.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.7510, 28.1289, 72.0000]]], dtype=torch.f

tensor(0) tensor([1])

tensor([[[ 0.3540, 26.8488, 32.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.8960, 38.4525, 24.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.8240, 29.8626, 70.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.5060, 46.0789, 34.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.0280, 30.0417, 54.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.5170, 26.6647, 47.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 1.4150, 28.1115, 59.0000]]], dtype=torch.float64)
tensor(0) tensor([7])

tensor([[[ 0.5600, 27.3988, 70.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.0780, 42.2480, 50.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.6150, 35.0000, 10.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.8670, 32.2503,  1.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 1.2740, 35.8249, 22.0000]]], dtype=torch.f

tensor(0) tensor([1])

tensor([[[ 1.7590, 37.5250,  6.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.0780, 33.2783, 22.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.5440, 26.2402, 38.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.9220, 31.3800, 16.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.7240, 29.9820, 22.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 0.6310, 26.5816, 38.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.5980, 36.9712, 74.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.4240, 45.4904, 61.0000]]], dtype=torch.float64)
tensor(0) tensor([14])

tensor([[[ 0.2270, 24.8296, 61.0000]]], dtype=torch.float64)
tensor(0) tensor([7])

tensor([[[ 0.8060, 27.5538, 27.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.4930, 31.3800, 76.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.6650, 29.9122, 43.0000]]], dtype=torch.

tensor(0) tensor([3])

tensor([[[ 1.8220, 27.1960,  8.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.7380, 43.1000, 40.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.5950, 29.6765, 73.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.3440, 29.0113, 31.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.6690, 32.9774, 34.0000]]], dtype=torch.float64)
tensor(0) tensor([8])

tensor([[[ 1.3530, 29.0262, 31.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.8690, 29.8042, 46.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.5470, 30.5069, 20.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 0.5890, 30.3946, 51.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.6810, 17.6500, 36.0000]]], dtype=torch.float64)
tensor(0) tensor([19])

tensor([[[ 0.9150, 32.0898, 77.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.5380, 34.3088, 33.0000]]], dtype=torch.

tensor([[[ 1.2130, 34.2092, 17.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.4150, 28.8231, 69.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.5370, 28.8760, 46.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 1.2110, 29.7064, 11.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.8110, 28.0643,  8.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.4940, 40.5502, 13.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 1.3980, 34.4863, 70.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.3790, 31.3800, 16.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.1550, 28.9198, 46.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.1830, 38.4510, 48.0000]]], dtype=torch.float64)
tensor(0) tensor([3])

tensor([[[ 1.7290, 41.8400,  2.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.3730, 35.6430, 32.0000]]], dtype=torch.float64)
tensor(0) tenso

tensor(0) tensor([2])

tensor([[[ 0.5760, 27.1565, 32.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.0580, 27.2409, 27.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 1.1310, 45.3964, 41.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.6510, 16.2723, 42.0000]]], dtype=torch.float64)
tensor(0) tensor([22])

tensor([[[ 0.6750, 21.6115,  5.0000]]], dtype=torch.float64)
tensor(0) tensor([12])

tensor([[[ 1.5680, 36.6949, 74.0000]]], dtype=torch.float64)
tensor(0) tensor([2])

tensor([[[ 1.3390, 32.5596, 66.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.5010, 30.0654, 31.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.4110, 25.9287,  1.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 1.1010, 28.5242, 25.0000]]], dtype=torch.float64)
tensor(0) tensor([1])

tensor([[[ 0.8980, 34.5702, 13.0000]]], dtype=torch.float64)
tensor(0) tensor([0])

tensor([[[ 1.0890, 38.4510, 48.0000]]], dtype=torch

KeyboardInterrupt: 

In [ ]:
elements = dc.element_phase_data.columns.values
measurements = ['C']

test_accuracies = pd.DataFrame(index=elements, columns=measurements)

# Hyperparameters
seq_len = 1
batch_size = 256

In [ ]:
# Train all networks

for element in elements:
    print('----------')
    print(element)
    for measurement in measurements:
        print('*****')
        print(measurement)
        # Create the dataset
        dc = DatasetCreator(elements=[element], splits=(0.8, 0.2), validation=False, seq_len=seq_len, measurement=measurement)
        train_dataset, test_dataset, val_dataset = dc.get_datasets()
        
        # Create the DataLoader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Create the network
        t = True
        net = PhaseClassifier(element=element, train=t, measurement=measurement, hidden_layers=2)
        
        # Train the network
        train(net, train_loader, batch_size, seq_len)
        
        # Save the network 
        path = r'C:\Users\danie\Documents\Montanuni\Masterarbeit\5 Programmcodes\packages\thermoclassifier\thermoclassifier\phases\models\new' + element + '_' + measurement + '.pth'
        torch.save(net, path)

In [77]:
path = r'C:\Users\danie\Documents\Montanuni\Masterarbeit\5 Programmcodes\packages\thermoclassifier\thermoclassifier\phases\models\new\PhaseClassifier.pth'
torch.save(net, path)

In [ ]:
# Test all networks

for element in elements:
    print('----------')
    print(element)
    for measurement in measurements:
        print('*****')
        print(measurement)
        # Create the dataset
        dc = DatasetCreator(elements=[element], splits=(0.8, 0.2), validation=False, seq_len=seq_len, measurement=measurement)
        train_dataset, test_dataset, val_dataset = dc.get_datasets()
        
        # Create the DataLoaders
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

        # Create the network
        t = False
        net = PhaseClassifier(element=element, train=t, measurement=measurement)
        
        # Test the network 
        test_accuracy = test(net, test_loader)
        
        # Save the test results
        test_accuracies[measurement][element] = test_accuracy